In [ ]:
#imports (same as tuto ML)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import OneHotEncoder # is this really needed ?
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score, train_test_split, GridSearchCV, PredefinedSplit

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

%matplotlib inline

We define long custom chart drawing and comparison functions we reuse a lot:

In [ ]:
#we use this a lot :D
def drawPies(u_rates, t_rates, labels, supertitle):
    """draws pretty comparative pie charts
    u_rates : rates for untreated group
    t_rates : rates for treated group
    lables: labels for values in rates
    supertitle: title of chart
    """
    fig = plt.figure(figsize=(7, 7))
    fig.suptitle(supertitle)
    
    plt.subplot(2,2,1)
    plt.pie(u_rates, labels = labels, autopct='%1.1f%%', shadow=True, startangle=90)
    plt.axis('equal')
    plt.title("control group")

    plt.subplot(2,2,2)
    plt.pie(t_rates, labels = labels, autopct='%1.1f%%', shadow=True, startangle=90)
    plt.axis('equal')
    plt.title("treated group")
    plt.show()

In [ ]:
def compare_groups(table) :
    """
    Compares groups in merged dataframe
    table : dataframe to compare treated and untreated in
    """
    columns = ['untreated', 'treated']
    #index = ['age', 'educ', 'black', 'hisp', 'married', 'no_degree']
    index = ['black', 'hisp', 'married', 'no_degree']
    result = pd.DataFrame(columns=columns, index=index)
    #result['untreated']['age'] = table['age_y'].mean()
    #result['untreated']['educ'] = table['educ_y'].mean()
    result['untreated']['black'] = table['black_y'].mean()
    result['untreated']['hisp'] = table['hispan_y'].mean()
    result['untreated']['married'] = table['married_y'].mean()
    result['untreated']['no_degree'] = table['nodegree_y'].mean()
    #result['treated']['age'] = table['age_x'].mean()
    #result['treated']['educ'] = table['educ_x'].mean()
    result['treated']['black'] = table['black_x'].mean()
    result['treated']['hisp'] = table['hispan_x'].mean()
    result['treated']['married'] = table['married_x'].mean()
    result['treated']['no_degree'] = table['nodegree_x'].mean()
    return result


#####mean isn't perfectly accurate!!!!!!!!!
#####it doesn't tell us about variance etc
####only use it for binary variables, then it gives us percentages
###use other function for interval data

# Question 1: Propensity score matching

We preform a naive data analysis using plots and numbers.

In [ ]:
#import the data set
lalonde_df = pd.read_csv('lalonde.csv')
#give a first look
lalonde_df.head()

### 1. A naive analysis

We assume that a naive researcher unfamiliar with observational studies would treat the data as if it was a randomized trial, not taking into consideration the hidden correlates.

We can easily imagine that the first thing he would do is split the salary (_['re78']_) data into 2 sets: treated and untreated.

In [ ]:
#masks to be used alot later
treated = (lambda x: x.treat == 1)
untreated = (lambda x: x.treat == 0)

#apply masks to get treated and untreated
treated_salary = lalonde_df[treated(lalonde_df)]['re78']
untreated_salary = lalonde_df[untreated(lalonde_df)]['re78']

**i - Describing the numbers**

We first look at the numbers to see how many subjects in each group we have an how the values are distributed

In [ ]:
lalonde_df.groupby('treat')['re78'].describe()

From the numbers above, we extract the following information from the data :

- The untreated group has more people.
- The untreated group's salaries have a higher mean.
- However, the max salary in the treated group is 3x higher! The 1st quartile is also two times higher on the treated group.
- Finally, we have that the second and third quartiles are higher in the untreated. Quartiles are more resistent to outliers, hence indicating that the control group is faring better.
- The interquartile distance is larger in the untreated set, as we have outliers in the set this is a better measure for 'variance'. Thus we say that the control group has a more heterogenous salary distribution.

**ii - Visualizing the data:**

We now plot the final salary data in a histogram to find the distribution of salaries of the two groups. We add weights so we can look at percentages instead of number of people in both bins, as the number of people in the two groups is not equal.

In [ ]:
plt.figure(figsize=(10, 4))
#define same bin size
bins = np.linspace(0, max(lalonde_df['re78']), 50)
#add weights to get percentages
plt.hist(untreated_salary, weights=np.ones(len(untreated_salary))/len(untreated_salary), alpha=.5 , bins=bins)
plt.hist(treated_salary, weights=np.ones(len(treated_salary))/len(treated_salary), alpha=.5, bins=bins)
plt.title('Histogram showing salary treated and untreated groups')
plt.legend(['untreated', 'treated'])
plt.xlabel('Yearly salary')
plt.ylabel('percentage of subjects')
plt.show()

##### First insights:

By looking at the graph, we see a very similar distribution for both functions, except that outliers are present in the treated group.

We also note that there are more relatively more subjects in the untreated group with a salary between 10k and 20k, while both groups have a similar ratio of subjects in the <10k section of the graph.

**iii - Boxplot:**

A boxplot will illustrate the above more consciely the 5 number summary we presented first. Note that we remove the outliers, as we consider them to not be representative of the outcome.

In [ ]:
def boxplot_treat_untreat(treated_salary, untreated_salary):
    """draws boxplot of distribution of salary """
    plt.boxplot([treated_salary, untreated_salary], labels=['treated', 'untreated'], showfliers=False)
    plt.title('Distribution of salary by treated and untreated')
    plt.ylabel('Salary')
    plt.show()

boxplot_treat_untreat(treated_salary, untreated_salary)

**Conclusion**:

If the treatment was effective, we should see that the treated group is more sucessful on average, as they were placed in program, wherease the untreated group was left to fend for themselves.

By merging all of the insights the researcher has drawn from the 3 steps of his analysis, he can conclude that **the treatment shows no effect**. The salary distributions are similar in both cases, indicating that the treatment isn't effective.

Additionaly, the treated group has in average a lower salary (except for the handful of people get lucky and find a good job). This is shown by the boxplot: the whiskers extend higher in the untreated group and the median and lower wisker as situated higher up. Since the difference is somewhat small this may just be due to chance.


### 2. A closer look at the data

After performing a simplistic analysis of the data ignoring underlying factors –such as race and education– that could influence the outcome, we start looking at the whole table assuming the other features will impact _['re78']_.

We split out analysis in by **categorical** and **intervall** data.

**i - Categorical data :**

Regarding categorical data, we should look at percentages present in each population, as the treated and control group have different sizes. Thus we define the rates for race, degree and mariage depending on each treatment to be able to compare them.

In [ ]:
categorical = ['black', 'hispan', 'married', 'nodegree']

#as the values are binary the mean is equal to the percentage of occurence
percentages = lalonde_df.groupby('treat')[categorical].mean()
percentages

##### a. Race ratios:

We will start with race. As we do not have numbers for "White" participants, we get the number of "Blacks" and "Hispanics" for each treatment group and substract the total. We then compare the rates of each race using pie charts.

In [ ]:
black_u, black_t = percentages['black']
hispan_u, hispan_t = percentages['hispan']
#there is no overlap in the hispan and black categories, 
#we assume people that are neither are white (which we checked, it is the case)
white_u, white_t = (1 - black_u - hispan_u, 1 - black_t - hispan_t)

In [ ]:
u_race_rates = [black_u, hispan_u, white_u]
t_race_rates = [black_t, hispan_t, white_t]
#give name to lable
race_labels = 'Black', 'Hispanic', 'White'

In [ ]:
drawPies(u_race_rates, t_race_rates, race_labels, 'Racial groups in percent by treatment')

We see that there are significantly more black subjects in the treatment group than in the untreated group. We also note that there are more hispanic people in the control group.

##### b. Degree ratios:

To have a better understanding of the difference of salaries, we also need to look at the level of education of the participants of each treatment.

In [ ]:
degree_u, degree_t = percentages['nodegree']

In [ ]:
#calculate rate for degree havers in treated and untreated group
u_degree_rates = [degree_u, 1 - degree_u]
t_degree_rates = [degree_t, 1 - degree_t]
degree_labels = 'Degree', 'No degree'

In [ ]:
#draw pie diagram
drawPies(u_degree_rates, t_degree_rates, degree_labels, 'Percentage of individuals with a degree, by treatment')

We see that the treated group is less educated, by a difference of over 10%.

##### c. Marriage ratios:
Finally, we look at the rates of married people among both groups as it is our last feature. 

In [ ]:
#married and unmarried by treatment
married_u, married_t = percentages['married']
not_married_u, not_married_t = (1 - married_u, 1 - married_t)

In [ ]:
u_marriage_rates = [married_u, not_married_u]
t_marriage_rates = [married_t, not_married_t]
mariage_labels = ['Married', 'Not married']

In [ ]:
drawPies(u_marriage_rates, t_marriage_rates, mariage_labels, 'Percentage of married individuals by treatment')

We again note that the treated group contains less married individuals.

##### d. Unemployment ratios:

Even though salaries are not categories but intervals, it is important to compare unemployment rates between both groups (which we define at categories, employed and unemployed). To get better insights, we will plot the years 1974 and 1975.

In [ ]:
#defining list of non binary variables
salaries = ['re74', 're75']
cat_salaries = lalonde_df.copy()

unemployed_labels = 'Employed', 'Unemployed'
#for each column draw a Boxplot
for sal in salaries:
    #get employed = 1 vs unemployed = 0
    cat_salaries[sal] = cat_salaries[sal].map(lambda x : 0 if x == 0 else 1)
    #get percentages
    u_employed, t_employed = cat_salaries.groupby('treat')[sal].mean()
    drawPies([1-u_employed, u_employed],[1-t_employed, t_employed], 
             unemployed_labels, 'Unemployment rates by treatment in 19'+sal[-2:] )

We see that our assumtion that both groups are balanced pre-treatment is wrong for unemplyment aswell.
There are much more unemployed people in the treated group than in the untreated group. 

**Conclusion:**

By looking at the categorical data, we can say that the underlying factors between the two groups are not similar at all.
The treated group is significantly more black, less educated, less employed and less married. All these factors influence employment and should be taken into consideration.

All these factors influence salary:
- due to racial inequelity (especialy given it was 1970) non-whites were likely to work in less well paying professionsand/or be underpayed
- better education gets bettr jobs
- people who already had jobs in 1974/5 are more likely to still have a job in 1978, and may have had a raise.
- marriage is an indicator of stability, which may influence job prospects. These individuals may have gotten married because they had a stable job. 

**ii - Interval data :**

We look at non binary data and the their distribution.


To do this we first do a box plot and relative fequency histogram for the intervall variables:

*note that we remove outliers from the box plot, this is justified as we care about the general distibution of values in the set.

In [ ]:
#we define a function that compares interval data, will be used again for 1.4

#defining list of non binary variables
intervals = ['age', 'educ', 're74', 're75'] #is reused in later fuction

def intervall_plots(df):
    """constructs boxplot and relative frequency histogram for data frame
    df: dataframe to be analyzed
    """
    #for each column draw a Boxplot
    for col in intervals:
        plt.figure(figsize=(10, 10))
        treated_ = df[treated(df)][col]
        untreated_ = df[untreated(df)][col]

        #boxplot
        plt.subplot(2,2,1)
        plt.title("Boxplot of " + col)
        plt.boxplot([untreated_, treated_], 
                    labels=['untreated', 'treated'], showfliers=False)
        plt.ylabel(col)

        #histogram
        plt.subplot(2,2,2)
        bins = np.linspace(min(df[col]), max(lalonde_df[col]), 50)
        plt.title("Relative frequency histogram of " + col)
        plt.ylabel('percentage')
        plt.xlabel(col)
        plt.hist(untreated_, weights=np.ones(len(untreated_))/len(untreated_), alpha=.5 , bins=bins)
        plt.hist(treated_, weights=np.ones(len(treated_))/len(treated_), alpha=.5, bins=bins)
        plt.legend(['untreated', 'treated'])
        
intervall_plots(lalonde_df)

We see that 
- pre treatment, the salaries are very unbalanced, the treatment group earning much less than the untreated group.
- there is a different age distribution in the two groups, the treated group being a bit younger, containg a lot of individuals in their 20's.
- their education level is fairly similar over all, but there are more well educated people in teh controll group, and years of education does not indicate the accuirement of a degree.

##### a. Evolution:

Even though it is useful to plot the salaries to see the difference between the years, it is much more useful to understand how the salary of each participant has changed over the years. 
It would be especially interesting to see if the in the outliers for the 3 years are connected.
To do so, we will visualize our data using a parallel plot. 

In [ ]:
#Implement parallel plot
from pandas.plotting import parallel_coordinates
parallel_coordinates(lalonde_df[untreated(lalonde_df)][['id','re74', 're75', 're78']], 'id', color='Blue', alpha=0.5)
parplot = parallel_coordinates(lalonde_df[treated(lalonde_df)][['id','re74', 're75', 're78']], 'id', color='Orange' , alpha=0.7)
#remove legend for readability
parplot.legend_.remove()
plt.title('Salary over time for each participant')
plt.xlabel('Year')
plt.ylabel('Annualy Salary')

Orange represents the treated group, blue the untreated group.

We see that:
- The treated group started out with a lower salary, as we can see the 15000 and above brachet is all blue.
- 75 was a bad year for everybody, treated or untreated, as we can see by the indent in the plot.
- The outliers are partialy people who were already well payed in 74, partialy people who 'made it'.
- There is a lot of movement up for the treated group between 75 and 78, we see a slight upwards movement for the untreated group.

#### b. Salary by race, education and marital status

We now look at salary in relation the categorical variables discussed before.

Above we hypothesized that these would influence the salary. Here we try to see if we can see such an influence on the salary.

**Plotting by race:**

In [ ]:
#defining mask for white
white = (lalonde_df['black'] == 0) & (lalonde_df['hispan'] == 0)

In [ ]:
years = ['re74', 're75', 're78']
for year in years:
    plt.boxplot([lalonde_df[white][year], lalonde_df[lalonde_df['black'] == 1][year],
                 lalonde_df[lalonde_df['hispan'] == 1][year]], 
                labels=['white', 'black', 'hispanic'])
    plt.title('salary distribution by race in 19'+year[-2:])
    plt.ylabel('annual earnings')
    plt.show()

We see that 
- there is a racial discrepancy in salary in our dataset
- the outliers are all black individuals

**By martial status and degree:**

In [ ]:
for year in years:
    #for every year we plot marriage
    plt.figure(figsize=(15,10))
    plt.subplot(2,2,1)
    plt.title('unmarried vs married salary in 19'+year[-2:])
    sns.boxplot(data=lalonde_df, x='married', y=year, hue='treat')
    plt.ylabel('salary')
    plt.xlabel('')
    plt.xticks(range(2),('unmarried', 'married'))
    
    #and degree
    plt.subplot(2,2,2)
    plt.title('degree vs nodegree salary in 19'+year[-2:])
    sns.boxplot(data=lalonde_df, x='nodegree', y=year, hue='treat')
    plt.xticks(range(2),('degree', 'no degree'))
    plt.xlabel('')
    plt.ylabel('salary')

We observe that:
- Married people make more than unmarried people, no matter the treatment or year
- People with degrees make more money that peopel with no degree, no matter the treatment or year
- Treatment seems even out the difference with the group, but not much between them. We see this by comparing the years pre treatment (1974/5) and the year after treatment (1978).

**Looking at age and salary**


It is well known that age my influence a persons prospect at finding a job and their potential salary, we thus map salary by age, and treatment for the 3 observed years.

In [ ]:
for year in years:
    sns.factorplot(data=lalonde_df, x='age', y=year, hue='treat',aspect=4, size=3)
    plt.title('salary by age and treatment in 19'+year[-2:])
    plt.yticks(np.linspace(0, 40000, 5))
    plt.show()

We see that:

- Pre-treatment, the treated group makes less that the controll group.
- Young people easely in the treatment group catch up on the control group post-treatment.
- Treatment group in general catches up, or at leats finds employment - the treatment seems to have an effect
- 75 bad year, low salary in general (look at y axis)

**Conclusion:**

By looking at the interval data, and more specificaly at the salaries of participants, we can say that the underlying conditions such as race, education marital status influence the salary.
As our two groups are not balanced, this interferes with our first analysis.

**iii - Correlation data :**

After working on each value alone, we want to understand how each value is (linearly) linked to others on each pair of features. We look at the pairplot, as correlation by itself does not give us any insights, as clearly the data is not linearly dependent.

We note that we can somewhat seperate the two groups, inicating that they are not the same.

It does not tell us alot, other than we can not see any linear dependence in the group.
We leave it as it is part of our analysis.

In [ ]:
sns.pairplot(lalonde_df[['treat', 're78']+intervals], markers='+', hue='treat')

### 3. A propsensity score model

As we have seen our naive analysis was preformed based the untrue assumption of a fair distribution in both sets.
This is not the case.
To create a fair set to use on our observational study, we calculate the propensity score based on the underlying factors before treatment:

[age, educ, hispan, black, nodegree, re74, re75]

In [ ]:
prop_table = lalonde_df.copy() #otherwise we modify lalonde_df when we modify prop_table

In [ ]:
#create our target and training data:

X = prop_table.iloc[:, 2:-1] #rows age to 're75'
y = prop_table.iloc[:, 1:2] #treated or not
y = np.ravel(y) #flatten array
print('First elements of Y : \n', y[0:5],'\nFirst elements of X\n', X[0:5])

In [ ]:
#define our model
logistic = LogisticRegression()
logistic.fit(X, y)
print('Accuracy of prediction: ',logistic.score(X, y))

In [ ]:
print("Example of prediction : ", logistic.predict(X[0:6]), ' reality :', y[0:6])
print('Example of prediction in percent : \n', logistic.predict_proba(X[0:6]))

In [ ]:
#get propensity scores, probability of "being a subject"
prop_table['propensity_scores'] = pd.Series(logistic.predict_proba(X)[:,1])

In [ ]:
prop_table.head()

We now use the propensity scores to find a matching.

### 4. Balancing the dataset via matching

We now try finding a matching based on the propensity scores to get a more balanced set.

Matching the two is an equivalent problem to find a matching in a bipartite graph

In [ ]:
import networkx as nx
B = nx.Graph()
#1. Creat graph with nodes as id
B.add_nodes_from(prop_table['id'])

In [ ]:
# 2. Add edges from each treated to each untreated subject
#    with weight on each node being the difference between the two
for row_i in prop_table[treated(prop_table)].iterrows():
    for row_j in prop_table[untreated(prop_table)].iterrows():
        B.add_edge(row_i[1]['id'],row_j[1]['id'], 
                   #-x to transform minimisation problem into maximisation problem
                   weight= 1 - np.abs(row_i[1].propensity_scores - row_j[1].propensity_scores))

In [ ]:
#3. Find matching
matching_dict = nx.max_weight_matching(B)

In [ ]:
print('Example matches:')
list(matching_dict.items())[:5]

In [ ]:
#get matching
matched = prop_table.copy()[prop_table['id'].isin(matching_dict)]
print('we have : ',len(matched)/2, ' matched subjects') #pairs appear in 2 order s ab and ba

#### looking at the outcome
We compare the outcome [re78] for treated and untreated groups

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(2,2,1)
sns.boxplot(data=matched, x='treat', y='re78')
plt.xticks(range(2), ('control', 'treatment'))
plt.title('Salary in 1978')
plt.subplot(2,2,2)
sns.boxplot(data=matched, x='treat', y='re78',  showfliers=False)
plt.xticks(range(2), ('control', 'treatment'))
plt.title('Salary in 1978, no outliers')
plt.show()

We now see that the treatment group's salary distribution in 1978 is slightly higher than the control groups.

#### Looking at the feature distrbution

We preforme a similary analysis to the one done in 1.2

In [ ]:
matched_percentages = matched.groupby('treat')[categorical].mean() #unbalanced
matched_percentages

We immediately see that for the categorical varibles, marriage and degree rates are more balanced, while the racial distribution in the two groups is still very skewed

We also look at intervall data again:

In [ ]:
intervall_plots(matched)

We note that they are all much more balanced than before, especially educ and re75.

### 5. Balancing the groups further


Based on your comparison of feature-value distributions from part 4, are you fully satisfied with your matching?
Would you say your dataset is sufficiently balanced?
If not, in what ways could the "balanced" dataset you have obtained still not allow you to draw valid conclusions?

Improve your matching by explicitly making sure that you match only subjects that have the same value for the problematic feature.
Argue with numbers and plots that the two groups (treated and control) are now better balanced than after part 4.

We noted above there are still way more black subjects in the treated group than in the untreated group.
Additionaly, we still have outliers in the treated group.

We try to balance the both groups by removing white subjects matched with black subjects.
We als balance the distribution of teh hispanic subjects in both groups
We first merge the two dataframes to have the matched subjects side by side:

In [ ]:
#mergin dataframe and adding propensity score difference column .. do we still need this?
matched['match'] = matched['id'].map(matching_dict)
balanced_match = matched[treated(matched)].merge(matched[untreated(matched)], left_on='id', right_on='match')
balanced_match['difference'] = abs(balanced_match['propensity_scores_x'] - balanced_match['propensity_scores_y'])
print('we have : ', len(balanced_match), ' matched subjects')
balanced_match.head()

We now remove the matched pairs that are white/black.

In [ ]:
#matches that are black/white hispanic/white missmatched
race_bool_1 = (balanced_match['black_x'] == 1) & (balanced_match['black_y'] == 0)
#this removes black hispanic pairs??? is this what you want??? it actuay doesn't change anything at all anyway
#tell me why...
#race_bool_2 = (balanced_match['black_x'] == 1) & (balanced_match['hispan_y'] == 1)
#other syntax didn't work
balanced_match = balanced_match.drop(balanced_match[race_bool_1].index)
#balanced_match = balanced_match.drop(balanced_match[race_bool_2].index)
print('we have : ', len(balanced_match), ' matched subjects')

We now check if the data set is more balanced:

In [ ]:
compare_groups(balanced_match)
#but now married is mismatched smh

In [ ]:
def plot_blanced_interval(df):
    df1 = df.iloc[:,:11]
    df2 = df.iloc[:, 13:-3]
    df2.columns = lalonde_df.columns
    df1.columns = lalonde_df.columns
    balanced_df = pd.concat([df1, df2])
    intervall_plots(balanced_df)
    
plot_blanced_interval(balanced_match)

In [ ]:
#matches that are black/white missmatched and have a large difference in propensity scores
marriage_bool_1 = (balanced_match['married_x'] == 1) & (balanced_match['married_y'] == 0)
marriage_bool_2 = (balanced_match['married_y'] == 1) & (balanced_match['married_x'] == 0)
balanced_match = balanced_match.drop(balanced_match[marriage_bool_1 | marriage_bool_2].index)
print('we have : ', len(balanced_match), ' matched subjects')

In [ ]:
compare_groups(balanced_match)

The match seems balanced blah blah

We look at intervall data and see blah blah

In [ ]:
plot_blanced_interval(balanced_match)

finaly we remove outliers from the lucky few from the treatment set

In [ ]:
#removing outliers - no need unless looking at mean
balanced_match = balanced_match.drop(balanced_match[balanced_match.re78_x > 30000].index)

### 6. A less naive analysis

Compare the outcomes (`re78`) between treated and control subjects, as you've done in part 1, but now only for the matched dataset you've obtained from part 5.
What do you conclude about the effectiveness of the job training program?

In [ ]:
boxplot_treat_untreat(balanced_match['re78_x'], balanced_match['re78_y'])

In [ ]:
stats = pd.concat([balanced_match.re78_y.describe(),balanced_match.re78_x.describe()], axis=1)
stats.columns = ['untreated', 'treated']
stats

After controlling for unerlying factors we see that the treated population fares better than the untreated population, which we can see as the overall distribution of the salary in the two groups is moved up.

# Question 2: Applied ML

First, we need to compute the TF-IDF features of our dataset, using a vectorizer. As we understood the question, what was asked was not to use any of the given datasets from sklearn, but to use all of the data. Thus, we do not use the train and test subsets given to us in sklearn, but will create our own such subsets, adding a validation subset.

Also note that we **remove the headers, footers and quotes**, as proposed in the <a href="http://scikit-learn.org/stable/datasets/twenty_newsgroups.html">sklearn tutorial</a> of the dataset, as to have something more realistic and without any of the metadata. 
This significantly decreases our accuracy, as we show later.

Note also that we did not use the *sklearn.datasets.fetch_20newsgroups_vectorized* function that returns the TF-IDF features directly, as it would defeat the purpose of the exercise.

In [ ]:
# create the TF-IDF vectorizer
tfidf = TfidfVectorizer()

In [ ]:
# Import the data we need to use the vectorizer on. Remove metadata as proposed by sci-kit tutorial
newsgroups_all = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroups_with_headera = fetch_20newsgroups(subset='all')

As asked in the question, before seperating in subsets, we will use the vectorizer on the complete set.

In [ ]:
# vectors is a sparse matrix
vectors = tfidf.fit_transform(newsgroups_all.data)

Now we need to seperate the dataset into three sets: train, test and validation.

In [ ]:
# first we seperate train from the rest. Random_state given to have a seed.
newsgroups_train, newsgroups_inter, vect_train, vect_inter = \
    train_test_split(newsgroups_all.target, vectors, test_size=0.2, random_state=1)

# then we seperate again to get validation and test seperately
newsgroups_test, newsgroups_valid, vect_test, vect_valid = \
    train_test_split(newsgroups_inter, vect_inter, test_size=0.5, random_state=1)

Now we need to train a random forest on our training set. For this, we will use the RandomForestClassifier, as it contains the parameters talked about in the exercise. But first, we need to ask ourselves what we want to set the parameters (*max_depth* and *n_estimators*) to.

According to the ADA course, we know that the number of trees will be in the 100's and the depth will be betweem 20 to 30. Thus for the training set, we set *n_estimators* to 100 and *max_depth* to 25.

For the predictions, we can't use the training set, as we just trained on it and thus would get very good results regardless. So prediction has to be on the validation set.

In [ ]:
# need to find estimators and depth first. We use random_state to have a seed again.
clf = RandomForestClassifier(n_estimators=100, max_depth=25, random_state=1)
clf.fit(vect_train, newsgroups_train)
pred = clf.predict(vect_valid)
metrics.f1_score(newsgroups_valid, pred, average='macro')

In [ ]:
metrics.accuracy_score(pred, newsgroups_valid)

As we can see, predictions aren't that great.

We try to fine tune on the validation set. We tried to use skikit's GridSearchCV function, but could not find the right parameters to make thefunction work. In fact, the best parameters were almost always the highest ones, yet the results of predictions stayed quite poor.

Thus, we decided to use a double for loop. Even if this type of computation is very heavy and takes a lot more time, it seemed the easier.

Please note that the fit takes a lot of time to compute, as there are a very large numbers of estimators.

In [ ]:
# We create a DF to be able to use all this data effectively
columns=['Depth', 'Estimators', 'Prediction']

In [ ]:
rows = []
# We made a few tries already to determine which values yield better results.
# We have a double for loop. Which is aweful and will take a lot of time, but necessary.
for i in range(30, 160, 10): #making sure the depth is not better above 30
    for j in range(300, 1100, 100): #step of 100 to not take too much time
        clf = RandomForestClassifier(n_estimators=j, max_depth=i)
        clf.fit(vect_train, newsgroups_train)
        pred = clf.predict(vect_valid)
        pred_result = metrics.accuracy_score(newsgroups_valid, pred)
        # uncomment next line to be able to know where computation is at
        print(i," ; ", j)
        rows.append([i, j, pred_result])

In [ ]:
clf = RandomForestClassifier(n_estimators=400, max_depth=100, random_state=0)
clf.fit(vect_train, newsgroups_train)
pred = clf.predict(vect_valid)
metrics.accuracy_score(newsgroups_valid, pred)

In [ ]:
clf = RandomForestClassifier(n_estimators=200, max_depth=200, random_state=0)
clf.fit(vect_train, newsgroups_train)
pred = clf.predict(vect_valid)
metrics.accuracy_score(newsgroups_valid, pred)

In [ ]:
all_predictions = pd.DataFrame(rows, columns=columns) # we get all the predictions in a dataframe
# we sort by best prediction
all_predictions = all_predictions.sort_values(by=['Prediction'], ascending=False)
# we show the best 10
all_predictions.head(5)

In [ ]:
# Saving the pickle. Do not run if we did not make a new pandas.
# all_predictions.to_pickle("all_predictions.pkl")
# read the pickle.
# all_predictions = pd.read_pickle("all_predictions.pkl")

90 	800 	0.680106

90 	900 	0.679576

90 	500 	0.678515

As we can see, te best resuts are when *n_estimators* is set around 800 to 900 and *max_depth* is set around 47. Note that because we have a bit of randomness, the results are not deterministic.

Now, we use thos numbers to create a random forest and train it:

In [ ]:
clf = RandomForestClassifier(n_estimators=800, max_depth=47, random_state=1)
clf.fit(vect_train, newsgroups_train)
pred = clf.predict(vect_valid)
metrics.f1_score(newsgroups_valid, pred, average='macro')

In [ ]:
metrics.accuracy_score(pred, newsgroups_valid)

Now we do a confusion matrix on the test set.

In [ ]:
pred_test = clf.predict(vect_valid)
cm = metrics.confusion_matrix(newsgroups_test, pred_test)
#normalize the matrix
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

To be able to read this matrix, lets put it into a DataFrame and add the right column names, then show the heatmap.

In [ ]:
df = pd.DataFrame(cm)
df.columns = newsgroups_all.target_names
df.index = newsgroups_all.target_names
plt.figure(figsize=(10, 6))
ax = sns.heatmap(df, cmap='OrRd')

Now, let us inspect the `feature_importances_` attribute of our random forest.

In [ ]:
#get names of features and their importance
features = pd.DataFrame(list(zip(tfidf.get_feature_names(), clf.feature_importances_)))
features = features.sort_values(by=[1], ascending=False)
features.head(10)

Let us also have an idea of hom many features are important or not by plotting them:

In [ ]:
plt.hist(clf.feature_importances_, bins=50)
plt.yscale('log')
plt.xlabel('Feature importance')
plt.ylabel('Number of features (log)');